In [11]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import os
# from keras.utils.np_utils import to_categorical
import numpy as np
from agents.knn import *
from utils import generate_id
from settings import settings

# def reduce_actions(actions):
#     s = np.unique(np.array([a['steer'] for a in actions]))
#     s.sort()
#     steer_values = np.unique(np.concatenate((
#         np.linspace(-1, -0.1, 15),
#         np.linspace(-0.1, 0 , 6),
#         np.linspace(0, 0.1, 6),
#         np.linspace(0.1, 1, 15),
#     )))
#     return [
#         {
#             **a,
#             'steer': steer_values[
#                 np.argmin(
#                     np.abs(
#                         steer_values - a['steer']
#                     )
#                  )
#             ]
#         } 
#         for a in actions
#     ]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
data = parse_dataset(
    '/home/kku/Dropbox/datasets/cg_track_2_2'
#     '/home/kku/Dropbox/important_logs',
#     8
)
test_size = 0.10

action_keys = [
    'steer',
    'accel',
    'brake'
]

state_keys = [
    'speedX',
    'speedY',
    'speedZ',
    'angle',
    'trackPos',
    'distFromStart',
    'track',
    'wheelSpinVel'
]

states, actions = extract_parameters(
    data,
    action_keys=action_keys,
    state_keys=state_keys
)
data_actions, actions = unify_actions(actions)
X, Y = sklearn.utils.shuffle(np.array(states), np.array(data_actions))
print(X.shape, Y.shape, np.unique(Y).shape)

(78138, 29) (78138,) (243,)


In [13]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)
print(X_train.shape)

(70324, 29)


In [14]:
%%time
scaler = sklearn.preprocessing.MinMaxScaler((-1,1))  
scaler.fit(X_train)  

# classifier = sklearn.ensemble.ExtraTreesClassifier(40, min_samples_leaf=4)
# classifier = sklearn.tree.DecisionTreeClassifier(min_samples_leaf=4)
classifier = sklearn.neural_network.MLPClassifier((200, 20))
# classifier = sklearn.svm.SVC(gamma='scale')

print(
    classifier.fit(
        scaler.transform(X_train),
        Y_train
    )
)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)
CPU times: user 30min 59s, sys: 11.1 s, total: 31min 11s
Wall time: 7min 48s


/home/kku/.miniconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [15]:
print(
    'Traning score: ', classifier.score(scaler.transform(X_train), Y_train), '\n'
    'Test score: ', classifier.score(scaler.transform(X_test), Y_test)
)
# sklearn.metrics.confusion_matrix(Y_test, classifier.predict(X_test))

Traning score:  0.7304049826517263 
Test score:  0.7019452265165088


In [16]:
def dump(path = 'saved_model'):
    print(f'Dumping model to: {path}')
    os.system(f'mkdir {path}')
    np.savez(
        f'{path}/parameters.npz',
        **{
            'actions': actions,
            'state_keys': state_keys
        }
    
    )
    joblib.dump(classifier, f'{path}/classifier')
    joblib.dump(scaler, f'{path}/scaler')    
path=f'models/{generate_id()}'
def remove_dump():
    print(os.system(f'rm -r {path}'))
dump(path)
    

Dumping model to: models/2019-01-08_20:23:50_799


In [7]:
asd = X_train[0]
asd.shape

(29,)

In [8]:
scaler.transform([asd])

array([[ 0.26908068,  0.08308917, -0.15247525, -0.04090376,  0.29898229,
         0.65966348, -0.30730879, -0.82477015, -0.2143005 , -0.33252836,
        -0.39859602, -0.42994428, -0.44616318, -0.46005572, -0.46980498,
        -0.47940498, -0.48885572, -0.49815522, -0.51091343, -0.5251204 ,
        -0.55066368, -0.59742886, -0.6628607 , -0.73127463,  0.31653813,
         0.27656307,  0.26714081,  0.28376876,  0.27909945]])

In [9]:
remove_dump()

0
